<a href="https://colab.research.google.com/github/tanmaylaud/biospec/blob/main/Sequence_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q wandb

     |████████████████████████████████| 1.8MB 34.1MB/s 
     |████████████████████████████████| 133kB 53.1MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 174kB 51.9MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/CSE291C00/Stage_1_Analysis_29_tissues_Project/'

/content/drive/MyDrive/CSE291C00/Stage_1_Analysis_29_tissues_Project


In [ ]:
import torch
import random
import numpy as np
import time
import torch.nn.functional as F
import math
import wandb

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
import pickle
with open('stomach_merged_large.pickle','rb') as f:
    merged = pickle.load(f)

In [ ]:
merged

,scan_id,PrecursorCharge,parent mass,Peptide,Protein,spectrum
0,3168,4,1437.74,R.EDSQRPGAHLTVK.K,tr|F8VYN5|F8VYN5_HUMAN,"(0, 52)\t0.017556602135300636\n (0, 56)\t0...."
1,3073,3,1078.59,R.EDSQRPGAHLTVK.K,tr|F8VYN5|F8VYN5_HUMAN,"(0, 29)\t0.2808007597923279\n (0, 31)\t0.00..."
2,6771,3,1079.60,K.ALVNQLHER.V,tr|D6R9R1|D6R9R1_HUMAN,"(0, 27)\t0.00489127216860652\n (0, 29)\t0.4..."
3,17822,3,1085.67,"R.(C,57.021)IIPSVIKR.A",tr|G3V524|G3V524_HUMAN,"(0, 29)\t0.5876671075820923\n (0, 30)\t0.02..."
4,3752,3,1087.53,Y.VNDAFGTAHR.A,sp|P00558|PGK1_HUMAN,"(0, 27)\t0.006831302773207426\n (0, 28)\t0...."
...,...,...,...,...,...,...
9244,32033,2,2564.17,K.GTWIHPEIDNPEYSPDPSIYAY.D,tr|K7EL50|K7EL50_HUMAN,"(0, 132)\t0.026059379801154137\n (0, 135)\t..."
9245,32281,2,2564.17,K.GTWIHPEIDNPEYSPDPSIYAY.D,tr|K7EL50|K7EL50_HUMAN,"(0, 161)\t0.03084501251578331\n (0, 163)\t0..."
9246,47874,3,3847.89,-.NVEDIELWLYEVEGHLASDDYGKDLTNVQNLQK.-,sp|Q13813|SPTN1_HUMAN,"(0, 225)\t0.007571768946945667\n (0, 280)\t..."
9247,30234,2,2568.17,"K.(G,3.999)TWIHPEIDNPEYSPDPSIYAY.D",tr|K7EL50|K7EL50_HUMAN,"(0, 135)\t0.022118493914604187\n (0, 157)\t..."


In [ ]:
merged['Peptide'].value_counts()

K.KYDEELEER.L                           689
-.VGAHAGEYGAEALER.-                     614
-.IQLVEEELDRAQER.-                      576
-.SYELPDGQVITIGNER.-                    530
-.TYFPHFDLSHGSAQVK.-                    503
                                       ... 
Y.SGSTNYNPSL(K,0.943)SR.V                 1
-.QSGESIDIITR.-                           1
-.FSTSGLSISGL(N,0.984)PLPNPSYLLPPR.-      1
K.RY(C,57.021)PNSVLVIIDVKPK.D             1
K.GIIDPTKVV.R                             1
Name: Peptide, Length: 77980, dtype: int64

In [ ]:
def contains_number(x):
  return any(char.isdigit() for char in x)

merged['contains_num'] = merged['Peptide'].apply(contains_number)

In [ ]:
filtered = merged[~merged['contains_num']]
filtered

,scan_id,PrecursorCharge,parent mass,Peptide,Protein,spectrum,contains_num
0,3168,4,1437.74,R.EDSQRPGAHLTVK.K,tr|F8VYN5|F8VYN5_HUMAN,"(0, 52)\t0.017556602135300636\n (0, 56)\t0....",False
1,3073,3,1078.59,R.EDSQRPGAHLTVK.K,tr|F8VYN5|F8VYN5_HUMAN,"(0, 29)\t0.2808007597923279\n (0, 31)\t0.00...",False
2,6771,3,1079.60,K.ALVNQLHER.V,tr|D6R9R1|D6R9R1_HUMAN,"(0, 27)\t0.00489127216860652\n (0, 29)\t0.4...",False
4,3752,3,1087.53,Y.VNDAFGTAHR.A,sp|P00558|PGK1_HUMAN,"(0, 27)\t0.006831302773207426\n (0, 28)\t0....",False
6,16090,3,1087.62,-.LRVDPVNFK.-,sp|P02008|HBAZ_HUMAN,"(0, 29)\t0.580760657787323\n (0, 31)\t0.052...",False
...,...,...,...,...,...,...,...
9241,36500,3,3837.66,V.SVNNEHNVANVDNNNGWDSWNSIWDYGNGFAATR.L,sp|Q9NS71|GKN1_HUMAN,"(0, 226)\t0.024037254974246025\n (0, 287)\t...",False
9243,31837,2,2564.17,K.GTWIHPEIDNPEYSPDPSIYAY.D,tr|K7EL50|K7EL50_HUMAN,"(0, 132)\t0.03335380181670189\n (0, 135)\t0...",False
9244,32033,2,2564.17,K.GTWIHPEIDNPEYSPDPSIYAY.D,tr|K7EL50|K7EL50_HUMAN,"(0, 132)\t0.026059379801154137\n (0, 135)\t...",False
9245,32281,2,2564.17,K.GTWIHPEIDNPEYSPDPSIYAY.D,tr|K7EL50|K7EL50_HUMAN,"(0, 161)\t0.03084501251578331\n (0, 163)\t0...",False


In [ ]:
filtered[filtered['parent mass'] <= 2050]

,scan_id,PrecursorCharge,parent mass,Peptide,Protein,spectrum,contains_num
0,3168,4,1437.74,R.EDSQRPGAHLTVK.K,tr|F8VYN5|F8VYN5_HUMAN,"(0, 52)\t0.017556602135300636\n (0, 56)\t0....",False
1,3073,3,1078.59,R.EDSQRPGAHLTVK.K,tr|F8VYN5|F8VYN5_HUMAN,"(0, 29)\t0.2808007597923279\n (0, 31)\t0.00...",False
2,6771,3,1079.60,K.ALVNQLHER.V,tr|D6R9R1|D6R9R1_HUMAN,"(0, 27)\t0.00489127216860652\n (0, 29)\t0.4...",False
4,3752,3,1087.53,Y.VNDAFGTAHR.A,sp|P00558|PGK1_HUMAN,"(0, 27)\t0.006831302773207426\n (0, 28)\t0....",False
6,16090,3,1087.62,-.LRVDPVNFK.-,sp|P02008|HBAZ_HUMAN,"(0, 29)\t0.580760657787323\n (0, 31)\t0.052...",False
...,...,...,...,...,...,...,...
8492,33263,2,2042.09,K.VFDEFKPLVEEPQNLIK.Q,tr|A0A087WWT3|A0A087WWT3_HUMAN,"(0, 97)\t0.024205707013607025\n (0, 109)\t0...",False
8502,33290,2,2044.93,-.VFDEFKPLVEEPQNLIK.-,sp|P02768|ALBU_HUMAN,"(0, 97)\t0.03420107066631317\n (0, 109)\t0....",False
8503,33069,2,2045.11,-.VFDEFKPLVEEPQNLIK.-,sp|P02768|ALBU_HUMAN,"(0, 97)\t0.014191227033734322\n (0, 109)\t0...",False
8505,35610,2,2046.07,K.VLGAFSDGLAHLDNLKGTFA.T,tr|A0A2R8Y7R2|A0A2R8Y7R2_HUMAN,"(0, 96)\t0.012251904234290123\n (0, 109)\t0...",False


In [ ]:
filtered = filtered[filtered['parent mass'] <= 2050]

In [ ]:
filtered = filtered.sample(frac=1.0, random_state=SEED)

In [ ]:
def process_peptide(peptide):
  result = ""
  if '.' in peptide:
    i = 0
    while(peptide[i]) != '.':
      i += 1
    i+=1
    while(peptide[i]) != '.':
      result += peptide[i]
      i+=1
    return result   
  else: 
    return peptide

filtered['Peptide'].apply(process_peptide)

5436           SRVVGNPFDSK
11669          LLEESLLSLIR
5117             SFEMLILGR
23665    ELTQDPAVSVALGQTVR
2428             TSVLHVMVK
               ...        
9072            LQHPDMLVTK
14213         NLDLDSIIAEVK
4184          DIISDTSGDFRK
9116       TDTSHHDQDHPTFNK
15111        LVHPGVAEVVFVK
Name: Peptide, Length: 121001, dtype: object

In [ ]:
filtered['Peptide'] = filtered['Peptide'].apply(process_peptide)

In [ ]:
filtered['Peptide'].value_counts()

IQLVEEELDRAQER      809
DSYVGDEAQSK         699
KYDEELEER           682
VGAHAGEYGAEALER     666
TYFPHFDLSHGSAQVK    618
                   ... 
RVHEEEDAGSQLIGFY      1
SAWLSEKEDAVNK         1
AGEIVVFKVEGR          1
TALAMGADR             1
LSSTQQSLAEK           1
Name: Peptide, Length: 26127, dtype: int64

In [ ]:
filtered_distinct = filtered.drop_duplicates(subset=['Peptide'])

In [ ]:
filtered_distinct['Peptide'].value_counts()

GLFISISDR             1
SLADAINTEFK           1
ISADSAIMNPASK         1
LLVISGIPTHLDEGVVR     1
ENDAVTIQVLNQLIQK      1
                     ..
KLTPEEEEILNK          1
IQIPRPDDPSNQIK        1
LLEMEEQAAFLVGSATPR    1
IVPNVLLEQGK           1
LSSTQQSLAEK           1
Name: Peptide, Length: 26127, dtype: int64

In [ ]:
filtered_distinct['Peptide_Length'] = filtered_distinct['Peptide'].apply(lambda x: len(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
filtered_distinct = filtered_distinct.reset_index(drop=True)

In [ ]:
from collections import defaultdict
distribution = defaultdict(lambda:0)
for peptide in filtered_distinct['Peptide']:
  for p in peptide:
    if p not in ['-','.']:
      distribution[p] +=1

distribution['C'] += 2000
distribution

defaultdict(<function __main__.<lambda>>,
            {'A': 25869,
             'C': 2094,
             'D': 18353,
             'E': 25556,
             'F': 12941,
             'G': 23597,
             'H': 8038,
             'I': 17489,
             'K': 19009,
             'L': 35909,
             'M': 5864,
             'N': 13659,
             'P': 19234,
             'Q': 17000,
             'R': 13441,
             'S': 23495,
             'T': 18492,
             'V': 23850,
             'W': 3231,
             'Y': 8735})

In [ ]:
from collections import OrderedDict

class Vocab:
  def __init__(self, init_token, end_token, pad_token):
    self.init_token = init_token
    self.end_token = end_token
    self.pad_token = pad_token
    self.stoi = OrderedDict()
    self.itos = OrderedDict() 

    self.stoi[init_token] = 0
    self.stoi[end_token] = 1
    self.stoi[pad_token] = 2
    self.itos[0] = init_token
    self.itos[1] = end_token
    self.itos[2] = pad_token
    self._NEXT = 3

  def _get_next(self):
    _next = self._NEXT 
    self._NEXT += 1
    return _next

  def __len__(self):
    return len(self.stoi)

class PField:
  def __init__(self, init_token = '<sos>', end_token = '<eos>', pad_token='<pad>'):
    self.vocab = Vocab(init_token, end_token, pad_token)

  def preprocess(self, sequence):
    sequence = sequence.strip()
    result = ""
    for s in sequence:
      if s not in ['.','-']:
        result+=s
    return result
    
  def build_vocab(self, dataframe, column):
    for peptide in dataframe[column].unique():
      peptide = self.preprocess(peptide)
      for p in peptide:
        if p not in self.vocab.stoi:
           _next_token = self.vocab._get_next()
           self.vocab.stoi[p] = _next_token
           self.vocab.itos[_next_token] = p

  def tokenize(self, sequence):
    sequence = self.preprocess(sequence)
    return [self.vocab.stoi[self.vocab.init_token]] + [self.vocab.stoi[s] for s in sequence] + [self.vocab.stoi[self.vocab.end_token]]

In [ ]:
FIELD = PField()
FIELD.build_vocab(filtered,'Peptide')

In [ ]:
weights = { FIELD.vocab.stoi[p]: 1/distribution[p] for p in distribution }

In [ ]:
weight_sum = sum(weights.values())

In [ ]:
weight_mean = np.mean(list(weights.values()))

In [ ]:
weights[0] = weight_mean
weights[1] = weight_mean
weights[2] = 0

In [ ]:
weights = { w: weights[w]/weight_sum for w in weights}
weights

{0: 0.05,
 1: 0.05,
 2: 0.0,
 3: 0.02146321019334736,
 4: 0.0375179022016737,
 5: 0.02114373683407531,
 6: 0.021370433677700394,
 7: 0.03691911000019739,
 8: 0.026218057787911832,
 9: 0.03896747728094399,
 10: 0.02747660456016434,
 11: 0.026528387789609982,
 12: 0.01404322380162901,
 13: 0.019732279053556744,
 14: 0.028834017010274812,
 15: 0.08599558722590317,
 16: 0.027270069407997848,
 17: 0.02966341902898213,
 18: 0.019493529842386493,
 19: 0.06273676579904157,
 20: 0.057730752546387654,
 21: 0.15607493763314645,
 22: 0.2408204983250698}

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
weights = [weights[w] for w in sorted(weights)]
weights = gaussian_filter1d(weights, sigma=2)
weights = gaussian_filter1d(weights, sigma=2)
weights_tensor = torch.Tensor(weights)

In [ ]:
weights_tensor = weights_tensor.divide(weights_tensor.sum())

In [ ]:
weights_tensor

tensor([0.0305, 0.0296, 0.0282, 0.0268, 0.0258, 0.0254, 0.0254, 0.0257, 0.0259,
        0.0260, 0.0262, 0.0267, 0.0279, 0.0299, 0.0328, 0.0367, 0.0424, 0.0508,
        0.0626, 0.0776, 0.0937, 0.1076, 0.1158])

In [ ]:
filtered_distinct['Peptide'][0]

'SRVVGNPFDSK'

In [ ]:
FIELD.tokenize(filtered_distinct['Peptide'][0])

[0, 3, 4, 5, 5, 6, 7, 8, 9, 10, 3, 11, 1]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
from torch.utils.data import Dataset
import random

class PeptideDataset(Dataset):
  def __init__(self, processed, field):
    self.data = processed.sort_values(by='Peptide_Length')
    self.data_len = len(processed)
    self.field = field
  
  def __len__(self):
    return self.data_len

  def __getitem__(self, idx):
    row = self.data.iloc[idx]
    mass, charge, spectrum, peptide = row['parent mass'], row['PrecursorCharge'], row['spectrum'].toarray(), row['Peptide']
    y = self.field.tokenize(peptide)
    y_len = len(y)
    return mass, charge, spectrum, y, y_len

class PaddingSorter:
    def __init__(self, device, field):
        self.device = device
        self.field = field
        
    def sort_batch(self, mass, charge, spectrum, targets, lengths):
        """
        Sort a minibatch by the length of the sequences with the longest sequences first
        return the sorted batch targes and sequence lengths.
        This way the output can be used by pack_padded_sequences(...)
        """
        seq_lengths, perm_idx = lengths.sort(0, descending=True)
        mass = mass[perm_idx]
        charge = charge[perm_idx]
        spectrum = spectrum[perm_idx]
        target_tensor = targets[perm_idx]
        return  mass, charge, spectrum, target_tensor, seq_lengths

    def pad_and_sort_batch(self, DataLoaderBatch):
        """
        DataLoaderBatch should be a list of (sequence, target, length) tuples...
        Returns a padded tensor of sequences sorted from longest to shortest, 
        """
        batch_size = len(DataLoaderBatch)
        batch_split = list(zip(*DataLoaderBatch))
        mass, charge, spectrum, y, y_len = batch_split[0], batch_split[1], batch_split[2], batch_split[3], batch_split[4]

        
        max_trg_length = max(y_len)
        padded_trgs = np.full((batch_size, max_trg_length),self.field.vocab.stoi[self.field.vocab.pad_token])
        for i, l in enumerate(y_len):
            padded_trgs[i, 0:l] = y[i][0:l]

        mass, charge, spectrum, y, y_len = self.sort_batch(torch.tensor(mass),
                                               torch.tensor(charge),
                                               torch.tensor(spectrum),
                                               torch.tensor(padded_trgs),
                                               torch.tensor(y_len))
        
        return {'x': {'mass': mass.float().to(self.device), 'charge':charge.float().to(self.device),'spectrum':spectrum.float().to(self.device)}, 
                'y': y.long().to(self.device),
                'y_len': y_len.long().to(self.device)}
    
    def __call__(self, batch):
        return self.pad_and_sort_batch(batch)

In [ ]:
dataset = PeptideDataset(filtered_distinct, FIELD)

In [ ]:
dataset.__len__()

26127

In [ ]:
print(dataset.__getitem__(112))

(820.4939999999998, 2, array([[0., 0., 0., ..., 0., 0., 0.]]), [0, 12, 11, 10, 12, 9, 6, 11, 1], 9)


In [ ]:
train_data,valid_data,test_data = torch.utils.data.random_split(dataset,[int(dataset.__len__()*0.8),
                                                                         int(dataset.__len__()*0.1),
                                                                         int(dataset.__len__()*0.1)+2],
                                                                generator=torch.Generator().manual_seed(SEED)
                                                                )

In [ ]:
print(train_data.__getitem__(10))

(1076.62, 2, array([[0., 0., 0., ..., 0., 0., 0.]]), [0, 12, 3, 3, 21, 5, 12, 12, 15, 11, 1], 11)


In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE = 1024
train_loader = DataLoader(train_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            collate_fn=PaddingSorter(device,FIELD))
valid_loader = DataLoader(valid_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            collate_fn=PaddingSorter(device,FIELD))
test_loader = DataLoader(test_data,
                            batch_size=BATCH_SIZE,
                            shuffle=False,
                            collate_fn=PaddingSorter(device,FIELD))

In [ ]:
next(iter(train_loader))['x']['charge'].shape[0]

1024

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
  def __init__(self):
    super(Encoder,self).__init__()
    
    self.convnet = nn.Sequential(nn.Conv1d(1, 32, 100, 20),
                                 nn.PReLU(),
                                 nn.MaxPool1d(50, stride=2),
                                 nn.Conv1d(32, 64, 5),
                                 nn.PReLU(),
                                 nn.MaxPool1d(2, stride=2))
    self.pos_embedding = nn.Embedding(2051, 1)

    self.linear = nn.Linear(640+1,256)

  def forward(self,spectrum, charge):
    spectrum = spectrum.to(device)
    batch_size = spectrum.shape[0]

    #create position tensor
    pos = torch.arange(0, spectrum.shape[-1]).unsqueeze(0).repeat(batch_size, 1).to(device)
        
    #pos = [0, 1, 2, 3, ..., src len - 1]
    pos_embedded = self.pos_embedding(pos)
    
    spectrum_with_pos = spectrum + pos_embedded.permute(0,2,1)
   
    convolved  = self.convnet(spectrum_with_pos.float())
    
    charge = torch.log(charge.float().to(device).unsqueeze(-1))
    
    convolved = convolved.view(convolved.size()[0],-1)

    concatenated = torch.cat([convolved, charge],dim=-1)
  
    return self.linear(concatenated)


class Decoder(nn.Module):

  def __init__(self, output_dim, embedding_dim, spectral_linear_dim, enc_hid_dim, dec_hid_dim, dropout):
    super(Decoder,self).__init__()
    self.output_dim = output_dim
    self.embedding_dim = embedding_dim
    self.hidden_dim = dec_hid_dim
    self.spectral_linear_dim = spectral_linear_dim
    self.embedding = nn.Embedding(output_dim, embedding_dim)
    self.rnn = nn.GRU(enc_hid_dim + embedding_dim + spectral_linear_dim, dec_hid_dim)
    self.fc_out = nn.Linear(dec_hid_dim, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, prev_output, encoder_output, hidden):
    
    embedded_prev_output = self.dropout(self.embedding(prev_output))
    
    rnn_input = torch.cat((embedded_prev_output, encoder_output), dim = -1)

    output, hidden = self.rnn(rnn_input.unsqueeze(0),hidden.unsqueeze(0))

    output = self.fc_out(output)

    return output.squeeze(0), hidden.squeeze(0)


class Seq2Seq(nn.Module):

  def __init__(self, encoder, decoder, device, freeze_encoder = True):
    super(Seq2Seq,self).__init__()
    self.encoder = encoder
    self.linear = nn.Linear(2051, decoder.spectral_linear_dim)
    self.decoder = decoder
    self.device = device
    if freeze_encoder:
      for p in self.encoder.parameters():
        p.requires_grad = False
    
  def forward(self, x, y, teacher_forcing_ratio = 0.5):
    charge, spectrum = x['charge'], x['spectrum']
    
    batch_size = charge.shape[0]
    trg_len = y.shape[1]

    trg_vocab_size = self.decoder.output_dim
        
    #tensor to store decoder outputs
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
    #input passed through pretrained encoder
    encoder_output = self.encoder(spectrum, charge)
                
    #first input to the decoder is the <sos> tokens
    input = y[:,0]

    spectrum = self.linear(spectrum).squeeze(1)

    encoder_output = torch.cat([encoder_output, spectrum], dim= -1)
    
    hidden = torch.zeros((batch_size, self.decoder.hidden_dim)).to(self.device)

                
    for t in range(1, trg_len):
      #insert input token embedding, previous hidden state, all encoder hidden states 
      #receive output tensor (predictions) and new hidden state
      
      output, hidden = self.decoder(input.squeeze(0), encoder_output, hidden)
            
      #place predictions in a tensor holding predictions for each token
      outputs[t] = output
            
      #decide if we are going to use teacher forcing or not
      teacher_force = random.random() < teacher_forcing_ratio
            
      #get the highest predicted token from our predictions
     
      top1 = output.argmax(1) 
            
      #if teacher forcing, use actual next token as next input
      #if not, use predicted token
      
      input = y[:,t] if teacher_force else top1
            
    return outputs

In [ ]:
ENCODER_NAME = 'encoder_siamese_cnn_stage2_2.pt'
OUTPUT_DIM = len(FIELD.vocab)
EMBEDDING_DIM = 256
SPECTRAL_LINEAR_DIM = 128
ENC_HID_DIM = 256
DEC_HID_DIM = 256
DROPOUT = 0.5
encoder = Encoder()
decoder = Decoder(OUTPUT_DIM, EMBEDDING_DIM,  SPECTRAL_LINEAR_DIM, ENC_HID_DIM, DEC_HID_DIM, DROPOUT)
model = Seq2Seq(encoder, decoder, device, freeze_encoder=True).to(device)

In [ ]:
for i, batch in enumerate(train_loader):
  x, y = batch['x'], batch['y']
  print(encoder(x['spectrum'], x['charge']))
  break

tensor([[ 0.2057,  0.1825, -0.1982,  ..., -0.2993,  0.7617, -0.0275],
        [ 0.2040,  0.1823, -0.2011,  ..., -0.3026,  0.7597, -0.0243],
        [ 0.2058,  0.1839, -0.1966,  ..., -0.2959,  0.7595, -0.0272],
        ...,
        [ 0.2071,  0.1827, -0.2004,  ..., -0.3000,  0.7611, -0.0281],
        [ 0.2054,  0.1835, -0.2001,  ..., -0.3020,  0.7626, -0.0269],
        [ 0.2047,  0.1838, -0.2003,  ..., -0.2988,  0.7635, -0.0257]],
       device='cuda:0', grad_fn=<AddmmBackward>)


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

model.apply(init_weights)

The model has 964,119 trainable parameters


Seq2Seq(
  (encoder): Encoder(
    (convnet): Sequential(
      (0): Conv1d(1, 32, kernel_size=(100,), stride=(20,))
      (1): PReLU(num_parameters=1)
      (2): MaxPool1d(kernel_size=50, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(32, 64, kernel_size=(5,), stride=(1,))
      (4): PReLU(num_parameters=1)
      (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (pos_embedding): Embedding(2051, 1)
    (linear): Linear(in_features=641, out_features=256, bias=True)
  )
  (linear): Linear(in_features=2051, out_features=128, bias=True)
  (decoder): Decoder(
    (embedding): Embedding(23, 256)
    (rnn): GRU(640, 256)
    (fc_out): Linear(in_features=256, out_features=23, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
model.encoder.load_state_dict(torch.load(ENCODER_NAME, map_location=device))

<All keys matched successfully>

In [ ]:
sample = next(iter(train_loader))
output  = model(sample['x'],sample['y'])
output.shape

torch.Size([35, 1024, 23])

In [ ]:
output[1:].shape

#trg = [trg len, batch size]
#output = [trg len, batch size, output dim]

torch.Size([34, 1024, 23])

In [ ]:
sample['y'][...,1:].shape

torch.Size([1024, 34])

In [ ]:
TRG_PAD_IDX = FIELD.vocab.stoi[FIELD.vocab.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX, 
                                weight=weights_tensor.to(device)
                                )

In [ ]:
N_EPOCHS = 40
CLIP = 1
RUN_NAME= 'seq2seq_1'
MODEL_NAME = RUN_NAME+'.pt'
def init_wandb(model,config,run_name):
    wandb.init(project="cse291c00_seq2seq",name=run_name,config=config)
    #try:
    #    wandb.watch(model)
    #except:
    #    print("Model already under watch!")

config = dict({
    'model_type':'gru',
    'seed':SEED,
    'n_epochs':N_EPOCHS,
    'clip':CLIP,
    'batch size': BATCH_SIZE,
    'trainable params': count_parameters(model),
})

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)

In [ ]:
def train(model, optimizer, criterion, train_dataloader, device, clip):
  model.train()
  epoch_loss = 0.0
  for i, batch in enumerate(train_dataloader):

    optimizer.zero_grad()

    x, y_true = batch['x'], batch['y'].to(device)

    output = model(x, y_true)
    
    output_dim = output.shape[-1]
        
    output = output[1:].view(-1, output_dim)

    y_true = y_true.permute(1,0)

    trg = y_true[1:].contiguous().view(-1)


    #print(trg.shape)
    #print(output.shape)

    loss = criterion(output,trg)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.decoder.parameters(), clip)

    optimizer.step()
      
    epoch_loss += loss.item()

  return epoch_loss/len(train_dataloader)

In [ ]:
def evaluate(model, criterion, valid_dataloader, device):
    
  model.eval()
  epoch_loss = 0

  with torch.no_grad():
    
    for i, batch in enumerate(valid_dataloader):

       x, y_true = batch['x'], batch['y'].to(device)
       
       output = model(x, y_true)
       
       output_dim = output.shape[-1]
        
       output = output[1:].view(-1, output_dim)

       y_true = y_true.permute(1,0)
       trg = y_true[1:].contiguous().view(-1)

       loss = criterion(output, trg)

       epoch_loss += loss.item()

    return epoch_loss/len(valid_dataloader)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
best_valid_loss = float('inf')
init_wandb(model,config,RUN_NAME)

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, optimizer, criterion, train_loader, device, CLIP)
    valid_loss = evaluate(model, criterion, valid_loader, device)
    scheduler.step(valid_loss)
    wandb.log({'epoch_loss':{
                    'epoch':epoch,
                    'train':{
                        'loss':train_loss
                    },
                    'eval':{
                        'loss':valid_loss
                    }
             }})
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_NAME)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.5f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

_runtime,403
_timestamp,1622001040
_step,39


_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Epoch: 01 | Time: 0m 10s
	Train Loss: 2.99821 | Train PPL:  20.050
	 Val. Loss: 2.89379 |  Val. PPL:  18.062
Epoch: 02 | Time: 0m 9s
	Train Loss: 2.86357 | Train PPL:  17.524
	 Val. Loss: 2.81100 |  Val. PPL:  16.627
Epoch: 03 | Time: 0m 10s
	Train Loss: 2.76966 | Train PPL:  15.953
	 Val. Loss: 2.72941 |  Val. PPL:  15.324
Epoch: 04 | Time: 0m 10s
	Train Loss: 2.73348 | Train PPL:  15.386
	 Val. Loss: 2.71324 |  Val. PPL:  15.078
Epoch: 05 | Time: 0m 10s
	Train Loss: 2.70599 | Train PPL:  14.969
	 Val. Loss: 2.69034 |  Val. PPL:  14.737
Epoch: 06 | Time: 0m 10s
	Train Loss: 2.67460 | Train PPL:  14.506
	 Val. Loss: 2.66449 |  Val. PPL:  14.361
Epoch: 07 | Time: 0m 10s
	Train Loss: 2.65495 | Train PPL:  14.224
	 Val. Loss: 2.67055 |  Val. PPL:  14.448
Epoch: 08 | Time: 0m 10s
	Train Loss: 2.64611 | Train PPL:  14.099
	 Val. Loss: 2.65416 |  Val. PPL:  14.213
Epoch: 09 | Time: 0m 9s
	Train Loss: 2.64995 | Train PPL:  14.153
	 Val. Loss: 2.67210 |  Val. PPL:  14.470
Epoch: 10 | Time: 0m 

In [ ]:
model.load_state_dict(torch.load(MODEL_NAME, map_location=device))

test_loss = evaluate(model, criterion, test_loader, device)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 2.559 | Test PPL:  12.928 |


In [ ]:
class Beam:
    def __init__(self, prob, token_index, hidden, prev = None):
        self.prob = prob
        self.token_index = token_index
        self.hidden = hidden
        self.prev = prev

    def flatten(self):
        beam_list = []
        curr = self
        while curr is not None:
            #print("Insert beam...")
            beam_list.insert(0,curr)
            curr = curr.prev
        return beam_list

In [ ]:
def beam_search_torch(model, device, hidden, encoder_outputs, beam_width, max_len, trg_field, beam_penalty):
    model.eval()
    initial_token_index = trg_field.vocab.stoi[trg_field.vocab.init_token]
    initial_beam = Beam(torch.tensor(1),initial_token_index, hidden)
    queue = []
    targets = []
    VOCAB_SIZE = len(trg_field.vocab)
    HIDDEN_SIZE = hidden.size(1)
    queue.append(initial_beam)
    while len(queue)!= 0 and max_len != 0:
        size = len(queue)
        candidates = torch.zeros(size, VOCAB_SIZE).to(device)
        hidden_states = torch.zeros(size, HIDDEN_SIZE).to(device)
        beams  = []
        #print(f"Current beam size:{size}")
        #print("Looking for candidates...")
        for i in range(0,size):
            beam = queue.pop()
            with torch.no_grad():
                trg_tensor = torch.LongTensor([beam.token_index]).to(device)
                #print(beam.hidden.view(1,-1).shape)
                output, hidden = model.decoder(trg_tensor, encoder_outputs, beam.hidden.view(1,-1))
                output = torch.nn.functional.log_softmax(output,-1)
            candidates[i] = output + beam.prob 
            hidden_states[i] = hidden
            beams.append(beam)
       
        probs, token_indices = torch.topk(candidates.view(-1), beam_width)
        #print(probs)
        beam_ids =  token_indices//VOCAB_SIZE
        indices = token_indices.fmod(VOCAB_SIZE)
        #print(indices)
        #print([trg_field.vocab.itos[index] for index in indices])
        #print(f"Found {len(candidates)} top candidates...")
        for i in range(0,beam_width):
            prev_beam = beams[beam_ids[i]]
            prob, token_index, hidden = probs[i],indices[i], hidden_states[beam_ids[i]]
            #print(trg_field.vocab.itos[token_index],end=',')
            beam = Beam(prob, token_index.item(), hidden, prev_beam)
            if token_index.item() == trg_field.vocab.stoi[trg_field.vocab.end_token]:
                targets.append(beam)
                #print(f"Adding beam to targets...")
            else:
                queue.append(beam)    
        max_len -= 1
                        
    if len(targets) ==0:
        print('WARNING: No <eos> found')
        for i in range(0,len(queue)):
            targets.append(queue.pop())
        
    best_beam = None
    max_prob  = float('-inf')
   
    #print('Len of targets:',len(targets))
    for i in range(0,len(targets)):
        
        beam = targets[i].flatten()
        #print([trg_field.vocab.itos[i.token_index] for i in beam[1:]])

        if beam[-1].prob > max_prob:
            max_prob = beam[-1].prob
            best_beam = beam
    #print(max_prob)       
    trg_tokens = [trg_field.vocab.itos[i.token_index] for i in best_beam]
    return trg_tokens

In [ ]:
def compute_beam(model,x, field, device, max_len = 30, beam_width = 10, beam_penalty = 1.0):
  model.eval()
  
  charge, spectrum = x['charge'], x['spectrum']
    
  trg_vocab_size = model.decoder.output_dim
  
  with torch.no_grad():      
    #input passed through pretrained encoder
    encoder_output = model.encoder(spectrum, charge)
    spectrum = model.linear(spectrum).squeeze(1)
    encoder_output = torch.cat([encoder_output, spectrum], dim= -1)           
    #first input to the decoder is the <sos> tokens
    
    hidden = torch.zeros((1, model.decoder.hidden_dim)).to(device)
      
  return beam_search_torch(model, device, hidden, encoder_output, beam_width, max_len, field, beam_penalty)

In [ ]:
def compute_sequence(model, x, field, device, max_len = 40):
  model.eval()
  
  charge, spectrum = x['charge'], x['spectrum']
    
  trg_vocab_size = model.decoder.output_dim
  
  outputs = []
  with torch.no_grad():      
    #input passed through pretrained encoder
    encoder_output = model.encoder(spectrum, charge)
    spectrum = model.linear(spectrum).squeeze(1)
    encoder_output = torch.cat([encoder_output, spectrum], dim= -1)           
    #first input to the decoder is the <sos> tokens
    
    hidden = torch.zeros((1, model.decoder.hidden_dim)).to(device)
    outputs.append(field.vocab.stoi[field.vocab.init_token])

    for t in range(0, max_len):
      #insert input token embedding, previous hidden state, all encoder hidden states 
      #receive output tensor (predictions) and new hidden state
      #print(encoder_output.shape)
      #print(input.shape)
      input = torch.tensor(outputs[-1]).to(device)
      #print(input)
      output, hidden = model.decoder(input.unsqueeze(0), encoder_output, hidden)
                
      #get the highest predicted token from our predictions
      trg_index = output.argmax(1).item()
      #print(trg_index.shape)
      outputs.append(trg_index)
      if trg_index == field.vocab.stoi[field.vocab.end_token]:
        break 
        
  return [field.vocab.itos[token] for token in outputs]

In [ ]:
example_loader = DataLoader(test_data,
                            batch_size=1,
                            shuffle=False,
                            collate_fn=PaddingSorter(device,FIELD))

In [ ]:
itera = iter(example_loader)

In [ ]:
example = next(itera)
src = example['x']
trg = [FIELD.vocab.itos[token.item()] for token in example['y'][0]]

print(f'src = {src}')
print(f'trg = {trg}')
print(f'prd = {compute_sequence(model, src, FIELD, device)}')

src = {'mass': tensor([1550.6899], device='cuda:0'), 'charge': tensor([2.], device='cuda:0'), 'spectrum': tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')}
trg = ['<sos>', 'Y', 'L', 'T', 'D', 'S', 'E', 'Y', 'T', 'E', 'G', 'S', 'T', 'G', 'K', '<eos>']
prd = ['<sos>', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'R', '<eos>']


In [ ]:
example = next(itera)
src = example['x']
trg = [FIELD.vocab.itos[token.item()] for token in example['y'][0]]

print(f'src = {src}')
print(f'trg = {trg}')
print(f'prd = {compute_beam(model, src, FIELD, device, max_len = 30, beam_width=5, beam_penalty=1)}')

src = {'mass': tensor([1079.5400], device='cuda:0'), 'charge': tensor([2.], device='cuda:0'), 'spectrum': tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')}
trg = ['<sos>', 'K', 'D', 'L', 'G', 'E', 'E', 'N', 'F', 'K', '<eos>']
prd = ['<sos>', 'T', 'F', 'E', 'E', 'H', 'H', 'E', 'K', '<eos>']


In [ ]:
max_score = 0
sequence = {}
for i, example in enumerate(example_loader):
  #print(example)
  src = example['x']
  trg = [FIELD.vocab.itos[token.item()] for token in example['y'][0]]
  trg = trg[1:-1]
  pred = compute_sequence(model, src, FIELD, device)
  pred = pred[1:-1]
  # print(f'src = {src}')
  # print(f'trg = {trg}')
  # print(f'prd = {pred}')

  score = bleu_score([pred], [[trg]], max_n = 2, weights = [0.5,0.5])*100
  sequence[score] = ["".join(trg),"".join(pred)]

In [ ]:
keys = sorted(sequence, reverse = True)

In [ ]:
import pandas as pd
result = pd.DataFrame(sequence).T.reset_index().sort_values(by="index",ascending=False)
result.columns = ['score', 'Target', 'Predicted']
results = result.head(20)

In [ ]:
results

,score,Target,Predicted
217,59.628481,LLEEYTQLAR,LLEEAQQLLR
303,57.772983,LEAALGEAK,LLEALLAK
192,52.440441,LIETELLQLTQLELK,LLLEEALQQQLQLLLK
104,45.643547,LQQLQMEK,AQQQLQAAK
87,45.291084,AKPAEAPAAAAPK,AAAAAAAAAAAAK
79,43.643579,NQLLAER,QQLLLLR
288,42.365927,LLLEFTDTSYEEK,LLLEEEYLYYYYY
55,42.257714,VLLEAAIR,LLLLAAAR
204,41.931397,LENLLLLDLQHNR,LLLELHLHLHLLAK
305,41.408979,AEEYEFLTPVEEAPK,ALYYEEEEAAAAAK


In [ ]:
results.to_csv('Sequence_Results.csv')

In [ ]:
sequence

[{'charge': tensor([2.], device='cuda:0'),
  'mass': tensor([1235.6700], device='cuda:0'),
  'spectrum': tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')},
 ['<sos>', 'L', 'L', 'E', 'E', 'Y', 'T', 'Q', 'L', 'A', 'R', '<eos>'],
 ['<sos>', 'L', 'L', 'E', 'E', 'A', 'Q', 'Q', 'L', 'L', 'R', '<eos>']]

In [ ]:
from torchtext.data.metrics import bleu_score
def calculate_bleu(data, FIELD, model, device, max_len = 30, beam_width = 1, beam_penalty = 0.75):
    trgs = []
    pred_trgs = []
    length_score = 0
    for i, example in enumerate(data):
        #print(example)
        _, charge, spectrum, trg, _ = example
        src = {'charge': torch.Tensor([charge]), 'spectrum': torch.Tensor(spectrum).to(device)}
        trg = [FIELD.vocab.itos[token] for token in trg]
        
        pred_trg = compute_beam(model, src, FIELD, device, max_len, beam_width, beam_penalty)

        pred_trgs.append(pred_trg)
        trgs.append([trg])
        length_score += int(np.abs(len(pred_trg)-len(trg)) <= 2)
        if i % 300 == 0:
          print(f'{i} Samples generated...')
        if i > 1000:
          print('Length score:',length_score*1.0*100/1001)
          break  
    #print(pred_trgs)
    return bleu_score(pred_trgs, trgs, max_n=3, weights=[0.33,0.33,0.33])*100,bleu_score(pred_trgs, trgs, max_n=2, weights=[0.5,0.5])*100,bleu_score(pred_trgs, trgs, max_n=1, weights=[1])*100


def calculate_bleu_random(data, FIELD):
    trgs = []
    pred_trgs = []
    lengths = []
    length_score = 0
    for example in test_data:
      lengths.append(example[-1])
    min_len, max_len = np.min(lengths), np.max(lengths)
    for i, example in enumerate(data):
        _, _, _, trg, _ = example
        trg = [FIELD.vocab.itos[token] for token in trg]
        
        pred_trg = [FIELD.vocab.itos[np.argmax(np.random.multinomial(1, weights_tensor.numpy()))] for i in range(0, np.random.randint(min_len, max_len+1))]

        pred_trgs.append(pred_trg)
        trgs.append([trg])
        length_score += int(np.abs(len(pred_trg)-len(trg)) <= 2)
        if i % 300 == 0:
          print(f'{i} Samples generated...')
        if i > 1000:
          print('Length score:',length_score*1.0*100/1001)
          break  
    #print(pred_trgs)
    return bleu_score(pred_trgs, trgs, max_n=3, weights=[0.33,0.33,0.33])*100,bleu_score(pred_trgs, trgs, max_n=2, weights=[0.5,0.5])*100,bleu_score(pred_trgs, trgs, max_n=1, weights=[1])*100

def calculate_bleu_random_biased(data, FIELD):
    trgs = []
    pred_trgs = []
    lengths = []
    length_score = 0
    for example in test_data:
      lengths.append(example[-1])
    min_len, max_len = np.min(lengths), np.max(lengths)
    for i, example in enumerate(data):
        _, _, _, trg, _ = example
        trg = [FIELD.vocab.itos[token] for token in trg]
        
        pred_trg = [FIELD.vocab.itos[np.argmax(np.random.multinomial(1, weights_tensor.numpy()))] for i in range(0, len(trg)+1)]

        pred_trgs.append(pred_trg)
        trgs.append([trg])
        length_score += int(np.abs(len(pred_trg)-len(trg)) <= 2)
        if i % 300 == 0:
          print(f'{i} Samples generated...')
        if i > 1000:
          print('Length score:',length_score*1.0*100/1001)
          break  
    #print(pred_trgs)
    return bleu_score(pred_trgs, trgs, max_n=3, weights=[0.33,0.33,0.33])*100,bleu_score(pred_trgs, trgs, max_n=2, weights=[0.5,0.5])*100,bleu_score(pred_trgs, trgs, max_n=1, weights=[1])*100


In [ ]:
calculate_bleu(test_data, FIELD, model, device, beam_width= 3)

0 Samples generated...
300 Samples generated...
600 Samples generated...
900 Samples generated...
Length score: 88.51148851148851


(6.609015011238808, 16.646854788855133, 33.575250925547294)

In [ ]:
calculate_bleu_random(test_data, FIELD)

0 Samples generated...
300 Samples generated...
600 Samples generated...
900 Samples generated...
Length score: 14.385614385614385


(1.4184435829520226, 6.749681383371353, 26.696643233299255)

In [ ]:
calculate_bleu_random_biased(test_data, FIELD)

0 Samples generated...
300 Samples generated...
600 Samples generated...
900 Samples generated...
Length score: 100.0999000999001


(1.4717801474034786, 7.457941770553589, 31.34995698928833)